In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
%pprint
%who

# https://www.kaggle.com/jyzaguirre/us-homicide-reports/version/2
homicides_df = load_csv('U.S._Homicide_Reports,_1980-2014_Homicides_between_1980_and_2014')

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
DATA_FOLDER	 ENCODING_TYPE	 SAVES_FOLDER	 attempt_to_pickle	 example_iterrows	 get_column_descriptions	 load_csv	 load_dataframes	 load_object	 
math	 os	 pd	 pickle	 save_dataframes	 store_objects	 


In [7]:

homicides_df.columns

Index(['Record ID', 'Agency Code', 'Agency Name', 'Agency Type', 'City',
       'State', 'Year', 'Month', 'Incident', 'Crime Type', 'Crime Solved',
       'Victim Sex', 'Victim Age', 'Victim Race', 'Victim Ethnicity',
       'Perpetrator Sex', 'Perpetrator Age', 'Perpetrator Race',
       'Perpetrator Ethnicity', 'Relationship', 'Weapon', 'Victim Count',
       'Perpetrator Count', 'Record Source'],
      dtype='object')

In [9]:

len(homicides_df['State'].unique())

51

In [32]:

wiki_url = 'https://en.wikipedia.org/wiki/'
us_pop_df = pd.DataFrame([])
year_set = set([1980, 1990, 2000])
for year in year_set:
    try:
        tables_url = '{}{}_United_States_Census'.format(wiki_url, year)
        tables_df_list = pd.read_html(tables_url)

        pop_df = tables_df_list[1].dropna(axis=0, how='all').dropna(axis=1, how='all')
        pop_df.columns = pop_df.iloc[0]
        pop_df = pop_df.iloc[1:].T
        pop_df.columns = pop_df.iloc[1]
        pop_df = pop_df.iloc[range(0, 3, 2)].T
        
        pop_df.columns = ['Rank_{}'.format(year), 'Population_{}'.format(year)]
        us_pop_df = pd.merge(left=us_pop_df, right=pop_df, how='outer',
                             left_index=True, right_index=True)
    except Exception as e:
        print(e)
us_pop_df = us_pop_df.dropna()

,column_name,dtype,count_blanks,count_uniques,count_zeroes,has_dates,only_integers
0,Rank_1980,object,0,51,0,False,NaN
1,Population_1980,object,0,51,0,True,True
2,Rank_1990,object,0,51,0,False,NaN
3,Population_1990,object,0,51,0,True,True
4,Rank_2000,object,0,51,0,False,NaN
5,Population_2000,object,0,51,0,False,True


In [35]:

dtype_df = get_column_descriptions(us_pop_df)
match_series = (dtype_df['only_integers'] == True)

In [36]:

for column_name in dtype_df[match_series]['column_name']:
    us_pop_df[column_name] = us_pop_df[column_name].map(lambda x: int(x))

In [128]:

year_set = set([1980, 1990, 2000])
max_year = max(set(homicides_df['Year'].unique()) & year_set)

In [38]:

homicides_df['Crime Type'].unique()

array(['Murder or Manslaughter', 'Manslaughter by Negligence'],
      dtype=object)

In [99]:

column_list = ['State', 'Year', 'Perpetrator Race']
homicides_df[column_list].sample(5).T.tail(15).T

,State,Year,Perpetrator Race
11787,Michigan,1980,Black
319735,Florida,1995,Unknown
224316,California,1991,Black
212973,New York,1990,White
528343,New York,2007,White


In [100]:

homicides_df.sample(5).T

,149256,442229,635360,215855,454226
Record ID,149257,442230,635361,215856,454227
Agency Code,MD01721,MOSPD00,PAPPD00,TNMPD00,FL05801
Agency Name,Prince George's County,St. Louis,Pittsburgh,Memphis,Sarasota
Agency Type,County Police,Municipal Police,Municipal Police,Municipal Police,Municipal Police
City,Prince George's,St. Louis city,Allegheny,Shelby,Sarasota
State,Maryland,Missouri,Pennsylvania,Tennessee,Florida
Year,1987,2002,2014,1990,2003
Month,March,November,January,February,July
Incident,3,3,4,2,618
Crime Type,Murder or Manslaughter,Murder or Manslaughter,Murder or Manslaughter,Murder or Manslaughter,Murder or Manslaughter


In [129]:

column_list = ['State', 'Perpetrator Race']
match_series = (homicides_df['Year'] == max_year)
homicides_groupby = homicides_df[match_series].groupby(by=column_list)
homicides_groupby['Perpetrator Count'].sum()

State          Perpetrator Race             
Alabama        Asian/Pacific Islander             0
               Black                              0
               Unknown                            0
               White                              2
Alaska         Asian/Pacific Islander             0
               Black                              0
               Native American/Alaska Native      1
               Unknown                            2
               White                              6
Arizona        Asian/Pacific Islander             2
               Black                              1
               Native American/Alaska Native      5
               Unknown                            2
               White                             28
Arkansas       Asian/Pacific Islander             0
               Black                             17
               Unknown                            0
               White                              7
California     Asia

In [40]:

wiki_url = 'https://en.wikipedia.org/wiki/'
tables_url = '{}Historical_racial_and_ethnic_demographics_of_the_United_States'.format(wiki_url)
tables_df_list = pd.read_html(tables_url)

In [133]:

# African–Americans as percentage of total population (1790–2010) by U.S. state
black_pop_df = tables_df_list[27]
black_pop_df.columns = black_pop_df.iloc[0]
black_pop_df = black_pop_df.iloc[1:].T
black_pop_df.columns = black_pop_df.iloc[0]
black_pop_df = black_pop_df.iloc[1:].T
black_pop_df.columns = [int(column_name) for column_name in black_pop_df.columns]
black_pop_df[max_year].sample(5)

State/territory
Kentucky           7.3%
North Carolina    21.6%
Virginia          19.6%
Wisconsin          5.7%
Texas             11.5%
Name: 2000, dtype: object

In [76]:

# Free Blacks as a percentage of the total Black (African-American)
# population by U.S. region and U.S. state between 1790 and 1860
tables_df_list[28].head(5)

,0,1,2,3,4,5,6,7,8
0,State/territory,1790,1800,1810,1820,1830,1840,1850,1860
1,United States,7.9%,10.8%,13.5%,13.2%,13.7%,13.4%,11.9%,11.0%
2,Northeast,40.1%,56.2%,73.5%,83.7%,97.8%,99.5%,99.8%,100.0%
3,Midwest,NaN,78.7%,52.4%,38.0%,37.7%,34.4%,35.5%,37.6%
4,South,4.7%,6.7%,8.5%,8.2%,8.4%,8.1%,7.0%,6.3%


In [116]:

# Percentage of population of Mexican origin (1910–1930)
# and of Hispanic/Latino origin (1940–2010) by U.S. state
tables_df_list[29].head(5)

,0,1,2,3,4,5,6,7,8,9,10,11
0,State/territory,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010
1,United States of America,0.4%,0.7%,1.2%,1.4%,NaN,NaN,4.7%,6.4%,9.0%,12.5%,16.3%
2,Northeast,0.0%,0.0%,0.0%,0.4%,NaN,NaN,4.1%,5.3%,7.4%,9.8%,12.6%
3,Midwest,0.0%,0.1%,0.3%,0.2%,NaN,NaN,1.5%,2.2%,2.9%,4.9%,7.0%
4,South,1.1%,1.7%,2.5%,1.9%,NaN,NaN,4.4%,5.9%,7.9%,11.6%,15.9%


In [81]:

# Asian and Pacific Islander % of Population by U.S. State (1860-2010)
tables_df_list[30].head(5).T.head(15).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,State/Territory,1860,1870,1880,1890,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990
1,United States of America,0.1%,0.2%,0.2%,0.2%,0.2%,0.2%,0.2%,0.2%,0.2%,0.2%,0.5%,0.8%,1.5%,2.9%
2,Alabama,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.1%,0.2%,0.5%
3,Alaska,NaN,NaN,NaN,7.1%,5.3%,3.8%,0.8%,0.8%,1.0%,NaN,0.8%,0.9%,2.0%,3.6%
4,Arizona,0.0%,0.2%,4.0%,1.3%,1.4%,0.8%,0.5%,0.6%,0.5%,0.4%,0.4%,0.5%,0.8%,1.5%


In [91]:

tables_df_list[40]

IndexError: list index out of range